In [1]:
%cd ..

/volatile/home/Zaccharie/workspace/fastmri-reproducible-benchmark


In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib nbagg
import os.path as op
import time

from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from keras_tqdm import TQDMNotebookCallback
import tensorflow as tf
from tqdm import tqdm_notebook

from data import MaskedUntouched2DSequence, MaskedUntouchedSingleSlice2DSequence
from pdnet import PDNet, InvShiftCropNet
from utils import keras_psnr, keras_ssim

/volatile/home/Zaccharie/workspace/fastmri-reproducible-benchmark/venv/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
# paths
train_path = '/media/Zaccharie/UHRes/singlecoil_train/singlecoil_train/'
val_path = '/media/Zaccharie/UHRes/singlecoil_val/'
test_path = '/media/Zaccharie/UHRes/singlecoil_test/'

In [4]:
n_samples_train = 34742
n_samples_val = 7135

n_volumes_train = 973
n_volumes_val = 199

In [5]:
# generators
AF = 4
# MaskedUntouched2DSequence, MaskedUntouchedSingleSlice2DSequence
train_gen = MaskedUntouched2DSequence(train_path, af=AF, inner_slices=16)
val_gen = MaskedUntouched2DSequence(val_path, af=AF)

In [6]:
run_params = {
    'n_primal': 3,
    'n_dual': 3,
    'n_iter': 5,
    'n_filters': 8,
#     'n_primal': 2,
#     'n_dual': 2,
#     'n_iter': 2,
#     'n_filters': 8,
}

n_epochs = 1
run_id = f'pdnet_subclassed_af{AF}_{int(time.time())}'
chkpt_path = f'checkpoints/{run_id}' + '-{epoch:02d}.hdf5'

In [7]:
chkpt_cback = ModelCheckpoint(chkpt_path, period=500, save_weights_only=True)
log_dir = op.join('logs', run_id)
tboard_cback = TensorBoard(
    log_dir=log_dir, 
    histogram_freq=0, 
    write_graph=True, 
    write_images=False, 
)
lr_on_plat_cback = ReduceLROnPlateau(monitor='val_loss', min_lr=5*1e-5, mode='auto', patience=3)
# fix from https://github.com/bstriner/keras-tqdm/issues/31#issuecomment-516325065
tqdm_cb = TQDMNotebookCallback(metric_format="{name}: {value:e}")
# tqdm_cb.on_train_batch_begin = tqdm_cb.on_batch_begin
# tqdm_cb.on_train_batch_end = tqdm_cb.on_batch_end
# tqdm_cb.on_test_begin = lambda x,y:None
# tqdm_cb.on_test_end = lambda x,y:None

W0806 17:55:45.971202 140508429580032 callbacks.py:875] `period` argument is deprecated. Please use `save_freq` to specify the frequency in number of samples seen.


In [8]:
# model = InvShiftCropNet(**run_params)
model = PDNet(**run_params)
model.compile(
    optimizer=Adam(lr=1e-2),
    loss='mean_absolute_error',
    metrics=['mean_squared_error', keras_psnr, keras_ssim],
)

In [9]:
model.fit_generator(
    train_gen, 
    steps_per_epoch=10, 
    epochs=n_epochs,
    validation_data=val_gen,
    validation_steps=25,
    verbose=2,
    callbacks=[tqdm_cb, tboard_cback, chkpt_cback, lr_on_plat_cback],
    max_queue_size=25,
#     use_multiprocessing=True,
    workers=25,
    shuffle=True,
)

Epoch 1/1000


W0806 17:55:50.961128 140508429580032 deprecation.py:323] From /volatile/home/Zaccharie/workspace/fastmri-reproducible-benchmark/venv/lib/python3.6/site-packages/tensorflow/python/autograph/impl/api.py:255: to_float (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.cast` instead.


10/10 - 50s - loss: 4.1251e-05 - mean_squared_error: 5.3749e-09 - keras_psnr: 14.2776 - keras_ssim: 0.2722 - val_loss: 2.9233e-05 - val_mean_squared_error: 2.4908e-09 - val_keras_psnr: 19.3870 - val_keras_ssim: 0.3451


Epoch 2/1000
10/10 - 30s - loss: 2.9372e-05 - mean_squared_error: 3.0021e-09 - keras_psnr: 18.6773 - keras_ssim: 0.3644 - val_loss: 2.1300e-05 - val_mean_squared_error: 1.2145e-09 - val_keras_psnr: 22.1491 - val_keras_ssim: 0.4146


Epoch 3/1000
10/10 - 30s - loss: 2.6145e-05 - mean_squared_error: 8.8683e-10 - keras_psnr: 19.9570 - keras_ssim: 0.3606 - val_loss: 1.8458e-05 - val_mean_squared_error: 8.8591e-10 - val_keras_psnr: 22.9230 - val_keras_ssim: 0.4291


...

Epoch 996/1000
10/10 - 28s - loss: 1.3399e-05 - mean_squared_error: 4.6129e-10 - keras_psnr: 24.4716 - keras_ssim: 0.4416 - val_loss: 9.9666e-06 - val_mean_squared_error: 2.3791e-10 - val_keras_psnr: 27.5660 - val_keras_ssim: 0.5328


Epoch 997/1000
10/10 - 28s - loss: 1.0710e-05 - mean_squared_error: 2.1578e-10 - keras_psnr: 26.3704 - keras_ssim: 0.5077 - val_loss: 9.9567e-06 - val_mean_squared_error: 2.3744e-10 - val_keras_psnr: 27.5765 - val_keras_ssim: 0.5331


Epoch 998/1000
10/10 - 28s - loss: 1.0471e-05 - mean_squared_error: 2.3370e-10 - keras_psnr: 27.1630 - keras_ssim: 0.5677 - val_loss: 9.9528e-06 - val_mean_squared_error: 2.3664e-10 - val_keras_psnr: 27.5782 - val_keras_ssim: 0.5330


Epoch 999/1000
10/10 - 28s - loss: 1.2239e-05 - mean_squared_error: 2.9022e-10 - keras_psnr: 26.2985 - keras_ssim: 0.5081 - val_loss: 9.9503e-06 - val_mean_squared_error: 2.3687e-10 - val_keras_psnr: 27.5840 - val_keras_ssim: 0.5332


Epoch 1000/1000
10/10 - 28s - loss: 1.6986e-05 - mean_squared_error: 6.8150e-10 - keras_psnr: 26.2501 - keras_ssim: 0.5839 - val_loss: 9.9543e-06 - val_mean_squared_error: 2.3721e-10 - val_keras_psnr: 27.5793 - val_keras_ssim: 0.5332



In [10]:
# # overfitting trials

# data = train_gen[0]
# val_data = val_gen[0]
# model.fit(
#     x=data[0], 
#     y=data[1], 
# #     validation_data=val_data, 
#     batch_size=data[0][0].shape[0], 
#     epochs=100, 
#     verbose=2, 
#     shuffle=False,
# )